In [2]:
import os
import torch
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.transforms.functional import resize as tvf_resize
from models.csrnet_mbv3 import MobileCSRNet

# ==== Config ====
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
IMAGE_DIR = "inference/test_images"
# MODEL_PATH = "build/csrnet_vgg_B.pth"
MODEL_PATH = "build/csrnet_mobile_B.pt"
SAVE_HEATMAP = True  # Set False jika tidak ingin simpan visualisasi density map
HEATMAP_DIR = "inference/heatmaps"

os.makedirs(HEATMAP_DIR, exist_ok=True)

# ==== Transform ====
# transform = transforms.Compose([
#     # transforms.Resize((384, 384)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406],
#                          std=[0.229, 0.224, 0.225]),

# ])

m_transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                            [0.229, 0.224, 0.225])
])

# ==== Load Model ====
model = MobileCSRNet().to(DEVICE)
model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False))
# checkpoint = torch.load(MODEL_PATH, map_location=DEVICE, weights_only=False)
# model.load_state_dict(checkpoint['state_dict'])
# print(f"MAE: {checkpoint['best_prec1']} - epoch: {checkpoint['epoch']}")
model.eval()

# ==== Predict ====
image_files = [f for f in os.listdir(
    IMAGE_DIR) if f.endswith(('.jpg', '.png'))]

for fname in image_files:
    path = os.path.join(IMAGE_DIR, fname)
    image = Image.open(path).convert('RGB')
    input_img = m_transform(image).unsqueeze(0).to(DEVICE)  # [1, 3, 384, 384]

    with torch.no_grad():
        output = model(input_img)  # [1, 1, H, W]
        density_map = output.squeeze().cpu().numpy()
        total_count = density_map.sum()

    print(f"{fname}: Estimated Count = {total_count:.2f}")

    # Optional: Save heatmap
    if SAVE_HEATMAP:
        # plt.figure(figsize=(4, 4))
        plt.imshow(density_map, cmap='jet')
        plt.axis('off')
        plt.title(f'Count: {total_count:.1f}')
        heatmap_path = os.path.join(
            HEATMAP_DIR, fname.replace('.jpg', '_heatmap.png'))
        plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0)
        plt.close()


34-peoples.jpg: Estimated Count = 54.78
271-peoples.jpg: Estimated Count = 175.04
93-peoples.jpg: Estimated Count = 103.10


In [1]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
import matplotlib.pyplot as plt

# ==== Config ====
IMAGE_DIR = "inference/test_images"
TFLITE_MODEL_PATH = "build/csrnet_mobile_B_float16.tflite"
HEATMAP_DIR = "inference/heatmaps_tflite"
SAVE_HEATMAP = True

os.makedirs(HEATMAP_DIR, exist_ok=True)

# ==== Load TFLite Model ====
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']  # e.g., [1, 512, 512, 3] or [1, 3, 512, 512]

is_channels_first = input_shape[1] == 3  # Check if input is NCHW or NHWC

# ==== Transform Function ====
def preprocess(image):
    image = image.resize((512, 512))
    image = np.array(image).astype(np.float32) / 255.0
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = (image - mean) / std
    if is_channels_first:
        image = np.transpose(image, (2, 0, 1))  # HWC -> CHW
    return np.expand_dims(image, axis=0).astype(np.float32)

# ==== Inference ====
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(('.jpg', '.png'))]

for fname in image_files:
    path = os.path.join(IMAGE_DIR, fname)
    image = Image.open(path).convert('RGB')
    input_tensor = preprocess(image)

    interpreter.set_tensor(input_details[0]['index'], input_tensor)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details[0]['index'])  # [1, 1, 64, 64]

    density_map = output.squeeze()
    total_count = density_map.sum()

    print(f"{fname}: Estimated Count = {total_count:.2f}")

    if SAVE_HEATMAP:
        plt.imshow(density_map, cmap='jet')
        plt.axis('off')
        plt.title(f'Count: {total_count:.1f}')
        heatmap_path = os.path.join(HEATMAP_DIR, fname.replace('.jpg', '_heatmap.png'))
        plt.savefig(heatmap_path, bbox_inches='tight', pad_inches=0)
        plt.close()


2025-06-09 14:23:07.305826: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-09 14:23:07.953597: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749453788.096985   80301 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749453788.135415   80301 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749453788.471020   80301 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

34-peoples.jpg: Estimated Count = 53.48
271-peoples.jpg: Estimated Count = 176.56
93-peoples.jpg: Estimated Count = 101.65
